**Environment Setup & Dependencies Installation**

In [1]:
!pip install -q datasets fsspec transformers accelerate scikit-learn faiss-cpu evaluate rouge_score huggingface_hub

import os, numpy as np, pandas as pd, torch, faiss, nltk
from datasets import load_dataset
from sklearn.metrics import f1_score, precision_score, recall_score, hamming_loss, classification_report
from transformers import (
    AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq,
    Seq2SeqTrainer, Seq2SeqTrainingArguments
)
from huggingface_hub import HfApi

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", DEVICE)

nltk.download('punkt', quiet=True)

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 72.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 118.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 98.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 5.4 MB/s eta 0:00:00
  

True

**RAG Mode Configuration (Enable/Disable Retrieval Augmentation)**

In [2]:
USE_RAG_TRAIN = True
USE_RAG_EVAL  = True
TEST_FRACTION = 0.10

**Load Datasets**

In [3]:
mimic_train_full = load_dataset("rntc/mimic-icd-visit", split="train", download_mode="force_redownload")
mimic_test_full  = load_dataset("rntc/mimic-icd-visit", split="test", download_mode="force_redownload")
icd10_data       = load_dataset("awacke1/ICD10-Clinical-Terminology", split="train", download_mode="force_redownload")

print(mimic_train_full.column_names)
print(icd10_data.column_names)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


README.md:   0%|          | 0.00/822 [00:00<?, ?B/s]

train-00000-of-00004.parquet:   0%|          | 0.00/249M [00:00<?, ?B/s]

train-00001-of-00004.parquet:   0%|          | 0.00/249M [00:00<?, ?B/s]

train-00002-of-00004.parquet:   0%|          | 0.00/248M [00:00<?, ?B/s]

...

Generating test split:   0%|          | 0/24458 [00:00<?, ? examples/s]

README.md:   0%|          | 0.00/101 [00:00<?, ?B/s]

ICD10Diagnosis.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/72750 [00:00<?, ? examples/s]

['subject_id', 'hadm_id', 'text', 'cleaned_text', 'icd_code', 'icd_descriptions', 'code_commons', 'code_rare']
['Code', 'Description']


**ICD-10 Knowledge Base & Retrieval System Setup**

In [8]:
from transformers import AutoTokenizer as HFTokenizer, AutoModel as HFModel

icd10_kb_texts = [f"{row['Code'].strip().upper()} - {row['Description'].strip()}"
            for row in icd10_data]

retriever_model_name = "emilyalsentzer/Bio_ClinicalBERT"
retriever_tokenizer = HFTokenizer.from_pretrained(retriever_model_name)
retriever_model = HFModel.from_pretrained(retriever_model_name).to(DEVICE).eval()


def compute_mean_pool(last_hidden_state, attention_mask):
    mask = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
    masked_output = last_hidden_state * mask
    summed_mask = masked_output.sum(dim=1)
    token_counts = torch.clamp(mask.sum(dim=1), min=1e-9)
    return summed_mask / token_counts

@torch.no_grad()
def encode_texts(texts, max_length=256, batch_size=64):
    if len(texts) == 0:
        return np.zeros((0, 768), dtype=np.float32)

    embeddings = []

    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i + batch_size]
        tokens = retriever_tokenizer(
            batch_texts,
            padding=True,
            truncation=True,
            max_length=max_length,
            return_tensors="pt"
        ).to(DEVICE)

        hidden_states = retriever_model(**tokens).last_hidden_state
        pooled_output = compute_mean_pool(hidden_states, tokens["attention_mask"])
        normalized = torch.nn.functional.normalize(pooled_output, p=2, dim=1)  # cosine-ready
        embeddings.append(normalized.cpu().numpy())

    return np.vstack(embeddings)

try:
    index = faiss.read_index("icd10_kb.index")
    print("Loaded FAISS index from disk.")
except Exception:
    print("Encoding ICD-10 KB…")
    kb_embs = encode_texts(icd10_kb_texts, max_length=256, batch_size=64)
    index = faiss.IndexFlatIP(kb_embs.shape[1])
    index.add(kb_embs)
    faiss.write_index(index, "icd10_kb.index")
    print("KB vectors indexed:", index.ntotal)

@torch.no_grad()
def retrieve_icd_snippets(query_text, top_k=5):
    query_embedding = encode_texts([query_text], max_length=256, batch_size=1)
    if query_embedding.shape[0] == 0:
        return []

    distances, indices = index.search(query_embedding, top_k)
    return [icd10_kb_texts[i] for i in indices[0]]

Encoding ICD-10 KB…
KB vectors indexed: 72750


**Data Preprocessing**

In [9]:
def preprocess_mimic_row(example, use_rag=True, top_k=5):
    cleaned_text = (example.get("cleaned_text") or "").replace("\n", " ").replace("\t", " ").strip().lower()

    if use_rag:
        try:
            retrieved_snippets = retrieve_icd_snippets(cleaned_text, top_k=top_k)
            rag_context = " ".join([f"[ICD] {snippet}" for snippet in retrieved_snippets])
            input_text = f"{cleaned_text}\n\nicd context: {rag_context}"
        except Exception:
            input_text = cleaned_text
    else:
        input_text = cleaned_text

    codes = example.get("icd_code", [])
    target_text = ",".join(sorted(set(str(code).strip().upper() for code in codes)))

    return {"input_text": input_text, "target_text": target_text}

def preprocess_mimic_batch(batch, use_rag=True, top_k=5):
    input_texts = []
    target_texts = []

    for text, codes in zip(batch.get("cleaned_text", []), batch.get("icd_code", [])):
        cleaned_text = (text or "").replace("\n", " ").replace("\t", " ").strip().lower()

        if use_rag:
            try:
                retrieved_snippets = retrieve_icd_snippets(cleaned_text, top_k=top_k)
                rag_context = " ".join([f"[ICD] {snippet}" for snippet in retrieved_snippets])
                input_text = f"{cleaned_text}\n\nicd context: {rag_context}"
            except Exception:
                input_text = cleaned_text
        else:
            input_text = cleaned_text

        target_text = ",".join(sorted(set(str(code).strip().upper() for code in codes)))

        input_texts.append(input_text)
        target_texts.append(target_text)

    return {"input_text": input_texts, "target_text": target_texts}

In [13]:
split_dataset = mimic_train_full.train_test_split(test_size=0.2, seed=42)

train_dataset = split_dataset["train"]
val_dataset = split_dataset["test"]

**Training and Deploy Model to Hugging Face Hub**

In [19]:
!pip install -U huggingface_hub
from huggingface_hub import login

login()

from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq
)
from huggingface_hub import HfApi
import torch

def run_training_with_rag(
    model_name: str,
    hf_repo_id: str,
    run_name: str,
    train_dataset,
    val_dataset,
    use_rag: bool = False,
    is_checkpoint: bool = False,
    output_dir_base: str = "./checkpoints"
):
    print(f"\n Training {model_name} | RAG: {use_rag}\n{'-'*60}")

    # Create repo on Hugging Face
    HfApi().create_repo(repo_id=hf_repo_id, private=False, exist_ok=True)

    # Preprocess using custom RAG logic
    train_dataset = train_dataset.map(lambda b: preprocess_mimic_batch(b, use_rag=use_rag), batched=True)
    val_dataset   = val_dataset.map(lambda b: preprocess_mimic_batch(b, use_rag=use_rag), batched=True)

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to("cuda" if torch.cuda.is_available() else "cpu")

    def tokenize_function(example):
        model_inputs = tokenizer(
            example["input_text"],
            max_length=512,
            padding="max_length",
            truncation=True
        )
        labels = tokenizer(
            text_target=example["target_text"],
            max_length=64,
            padding="max_length",
            truncation=True
        )
        model_inputs["labels"] = labels["input_ids"]
        return model_inputs

    tokenized_train = train_dataset.map(tokenize_function, batched=True, remove_columns=train_dataset.column_names)
    tokenized_val   = val_dataset.map(tokenize_function, batched=True, remove_columns=val_dataset.column_names)

    data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

    output_dir = f"{output_dir_base}/{model_name.replace('/', '')}_{'rag' if use_rag else 'plain'}"

    training_args = Seq2SeqTrainingArguments(
        output_dir=output_dir,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=3,
        logging_steps=100,
        eval_strategy="epoch",
        save_strategy="steps",
        save_steps=500,
        save_total_limit=2,
        predict_with_generate=True,
        fp16=torch.cuda.is_available(),
        report_to="wandb",
        run_name=run_name
    )

    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_val,
        tokenizer=tokenizer,
        data_collator=data_collator
    )

    trainer.train(resume_from_checkpoint=is_checkpoint)

    # Push to Hugging Face Hub
    model.save_pretrained(hf_repo_id, push_to_hub=True)
    tokenizer.save_pretrained(hf_repo_id, push_to_hub=True)
    print(f"Uploaded to: https://huggingface.co/{hf_repo_id}")

In [20]:
# T5-small without RAG
run_training_with_rag(
    model_name="t5-small",
    hf_repo_id="ShlokaTrivedi/mimic-icd-t5-small-plain",
    run_name="t5_small_plain",
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    use_rag=False,
    is_checkpoint=False
)

# T5-small with RAG
run_training_with_rag(
    model_name="t5-small",
    hf_repo_id="ShlokaTrivedi/mimic-icd-t5-small-rag",
    run_name="t5_small_rag",
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    use_rag=True,
    is_checkpoint=False
)

# BART-base without RAG
run_training_with_rag(
    model_name="facebook/bart-base",
    hf_repo_id="ShlokaTrivedi/mimic-icd-bart-plain",
    run_name="bart_plain",
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    use_rag=False,
    is_checkpoint=False
)

# BART-base with RAG
run_training_with_rag(
    model_name="facebook/bart-base",
    hf_repo_id="ShlokaTrivedi/mimic-icd-bart-rag",
    run_name="bart_rag",
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    use_rag=True,
    is_checkpoint=False
)


 Training t5-small | RAG: False
------------------------------------------------------------


Map:   0%|          | 0/62611 [00:00<?, ? examples/s]

Map:   0%|          | 0/15653 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

...

/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3909: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...imic-icd-bart-rag/model.safetensors:   1%|          | 2.89MB /  558MB            

Uploaded to: https://huggingface.co/ShlokaTrivedi/mimic-icd-bart-rag


**Prepare test set with retrieval-augmented context for ICD code prediction**

In [21]:
from datasets import load_dataset

test_data = load_dataset("rntc/mimic-icd-visit", split="test").shuffle(seed=42).select(range(500))

def preprocess_mimic(batch, use_rag=False, top_k=5):
    input_texts, target_texts = [], []

    for text, codes in zip(batch["cleaned_text"], batch["icd_code"]):
        cleaned = text.replace("\n", " ").replace("\t", " ").strip().lower()

        if use_rag:
            try:
                retrieved_snippets = retrieve_icd_snippets(cleaned, top_k=top_k)
                rag_context = " ".join([f"[ICD] {s}" for s in retrieved_snippets])
                cleaned += f"\n\nicd context: {rag_context}"
            except:
                pass

        label = ",".join(sorted(set(code.strip().upper() for code in codes)))
        input_texts.append(cleaned)
        target_texts.append(label)

    return {"input_text": input_texts, "target_text": target_texts}

**Comparative Model Evaluation: RAG vs Non-RAG Performance**

In [27]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq

model_repo_map = {
    ("t5-small", False): "ShlokaTrivedi/mimic-icd-t5-small-plain",
    ("t5-small", True):  "ShlokaTrivedi/mimic-icd-t5-small-rag",
    ("facebook/bart-base", False): "ShlokaTrivedi/mimic-icd-bart-plain",
    ("facebook/bart-base", True):  "ShlokaTrivedi/mimic-icd-bart-rag"
}

results_table = []

for (model_name, use_rag), repo_id in model_repo_map.items():
    print(f"\nEvaluating: {repo_id}")
    model_tag = f"{model_name.replace('/', '-')}-{'rag' if use_rag else 'plain'}"

    processed_test = test_data.map(lambda b: preprocess_mimic(b, use_rag=use_rag), batched=True)

    tokenizer = AutoTokenizer.from_pretrained(repo_id)
    model = AutoModelForSeq2SeqLM.from_pretrained(repo_id).to(DEVICE)

    def tokenize(example):
        model_inputs = tokenizer(
            example["input_text"],
            max_length=512,
            padding="max_length",
            truncation=True
        )
        labels = tokenizer(
            text_target=example["target_text"],
            max_length=64,
            padding="max_length",
            truncation=True
        )
        model_inputs["labels"] = labels["input_ids"]
        return model_inputs

    tokenized = processed_test.map(tokenize, batched=True)

    trainer = Seq2SeqTrainer(
        model=model,
        tokenizer=tokenizer,
        args=Seq2SeqTrainingArguments(
            output_dir="./eval_tmp",
            per_device_eval_batch_size=8,
            predict_with_generate=True,
            do_train=False,
            do_eval=False,
            report_to="none"
        ),
        data_collator=DataCollatorForSeq2Seq(tokenizer, model=model)
    )

    results = trainer.predict(tokenized)
    decoded_preds = tokenizer.batch_decode(results.predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(results.label_ids, skip_special_tokens=True)

    def to_label_set(label_list):
        return [set(item.split(',')) for item in label_list]

    true_labels = to_label_set(decoded_labels)
    pred_labels = to_label_set(decoded_preds)

    all_labels = sorted(set().union(*true_labels, *pred_labels))
    label2id = {label: i for i, label in enumerate(all_labels)}

    def binarize(label_sets):
        bin = []
        for label_set in label_sets:
            row = [0] * len(label2id)
            for label in label_set:
                if label in label2id:
                    row[label2id[label]] = 1
            bin.append(row)
        return bin

    y_true = binarize(true_labels)
    y_pred = binarize(pred_labels)

    # Metrics
    h_loss = hamming_loss(y_true, y_pred)
    f1_micro = f1_score(y_true, y_pred, average='micro')
    f1_macro = f1_score(y_true, y_pred, average='macro')
    precision = precision_score(y_true, y_pred, average='micro')
    recall = recall_score(y_true, y_pred, average='micro')

    print("Hamming Loss:", h_loss)
    print("Micro F1 Score:", f1_micro)
    print("Macro F1 Score:", f1_macro)
    print("Precision:", precision)
    print("Recall:", recall)

    results_table.append({
        "model": model_tag,
        "hamming_loss": h_loss,
        "f1_micro": f1_micro,
        "f1_macro": f1_macro,
        "precision": precision,
        "recall": recall
    })

    pd.DataFrame({
        "input_text": processed_test["input_text"],
        "true_icd_codes": processed_test["target_text"],
        "predicted_icd_codes": decoded_preds
    }).to_csv(f"predictions_{model_tag}.csv", index=False)

    # Save final table
    results_df = pd.DataFrame(results_table)
    results_df.to_csv("combined_model_metrics.csv", index=False)
    print("\n Combined metrics saved to: combined_model_metrics.csv")

    print("\n Final Evaluation Table:")
    print(results_df)


Evaluating: ShlokaTrivedi/mimic-icd-t5-small-plain


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

/tmp/ipython-input-1613442941.py:39: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Hamming Loss: 0.009135583185316756
Micro F1 Score: 0.14948737735641054
Macro F1 Score: 0.005818178195015174
Precision: 0.2314783202458177
Recall: 0.1103874959296646

 Combined metrics saved to: combined_model_metrics.csv

 Final Evaluation Table:
            model  hamming_loss  f1_micro  f1_macro  precision    recall
0  t5-small-plain      0.009136  0.149487  0.005818   0.231478  0.110387

Evaluating: ShlokaTrivedi/mimic-icd-t5-small-rag


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

...

/tmp/ipython-input-1613442941.py:39: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Hamming Loss: 0.007809360730593607
Micro F1 Score: 0.23641031365107712
Macro F1 Score: 0.055561449447241826
Precision: 0.37727110794442464
Recall: 0.17213914174252276

 Combined metrics saved to: combined_model_metrics.csv

 Final Evaluation Table:
                      model  hamming_loss  f1_micro  f1_macro  precision  \
0            t5-small-plain      0.009136  0.149487  0.005818   0.231478   
1              t5-small-rag      0.009127  0.149419  0.006064   0.233322   
2  facebook-bart-base-plain      0.007809  0.236410  0.055561   0.377271   

     recall  
0  0.110387  
1  0.109899  
2  0.172139  

Evaluating: ShlokaTrivedi/mimic-icd-bart-rag


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

/tmp/ipython-input-1613442941.py:39: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Hamming Loss: 0.007703577512776831
Micro F1 Score: 0.2432221354457213
Macro F1 Score: 0.058906253324003705
Precision: 0.3877623621487015
Recall: 0.1771781534460338

 Combined metrics saved to: combined_model_metrics.csv

 Final Evaluation Table:
                      model  hamming_loss  f1_micro  f1_macro  precision  \
0            t5-small-plain      0.009136  0.149487  0.005818   0.231478   
1              t5-small-rag      0.009127  0.149419  0.006064   0.233322   
2  facebook-bart-base-plain      0.007809  0.236410  0.055561   0.377271   
3    facebook-bart-base-rag      0.007704  0.243222  0.058906   0.387762   

     recall  
0  0.110387  
1  0.109899  
2  0.172139  
3  0.177178  
